Examples to load files and select variables by SQL  
Created by Zexuan Xu 

In [2]:
import pandas as pd
import sqlite3
from sqlalchemy.engine import create_engine

Create a databas called avito.db and add train.csv into the database

In [3]:
df = pd.read_csv('train.csv')
avito = sqlite3.connect("avito.db")
df.to_sql('train', avito)

ValueError: Table 'train' already exists.

In [4]:
df = pd.read_sql('SELECT * FROM train', avito)

Select the item_id term only

In [6]:
item = pd.read_sql('SELECT item_id FROM train', avito)


SELECT all items with price > 10000

In [9]:
price = pd.read_sql('SELECT * FROM train WHERE price > 10000 ', avito)
price

,index,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797
1,6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Сантехника и сауна,None,None,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125,2017-03-23,Private,0330f6ac561f5db1fa8226dd5e7e127b5671d44d075a98...,2823.0,0.00000
2,10,d5480bb4a6e4,d46200940f37,Ханты-Мансийский АО,Ханты-Мансийск,Недвижимость,Квартиры,Сдам,На длительный срок,1,"1-к квартира, 25 м², 2/2 эт.",Сдается однокомнатная мебелированная квартира ...,18000.0,6,2017-03-25,Private,645d3fb949cb116a00c596ca1e168d8c5ddd21cdeacc5a...,1396.0,0.58853
3,16,7896ef8fe482,007c6bd858cb,Краснодарский край,Краснодар,Транспорт,Автомобили,С пробегом,Chevrolet,Lanos,"Chevrolet Lanos, 2008","Кондиционер, гидроусилитель руля, электрически...",140000.0,201,2017-03-18,Company,5244375e00cb2baefa50f4b01156cc492f8733b93c6cba...,1118.0,0.32100
4,19,0658628930d4,27c2d83b766e,Белгородская область,Белгород,Недвижимость,"Дома, дачи, коттеджи",Сниму,Коттедж,На длительный срок,Сниму коттедж,Семейная пара из двух человек снимет коттедж с...,100000.0,5,2017-03-28,Private,None,NaN,0.10342
5,20,ea12aec32ec3,06ee4ff4e99c,Новосибирская область,Бердск,Недвижимость,Квартиры,Продам,2,Вторичка,"2-к квартира, 44 м², 5/5 эт.",Дом находиться внутри квартала./\nПластиковые ...,1748000.0,461,2017-03-17,Company,e9fb67b3d22cbf7cf569337780aea8a5c1ddac8b4b6b74...,2011.0,0.00000
6,22,de310e6aae86,0178e6308f38,Челябинская область,Магнитогорск,Транспорт,Автомобили,С пробегом,Ford,Focus,"Ford Focus, 2005",Полная комплектация. ГУР. 4 эсп. Подогрев сид...,249000.0,12,2017-03-20,Private,1c873f41cea87bc32f52d0a99cf5ff6a33305f0542b7f7...,2264.0,0.10334
7,33,c559542570bd,1446906a094c,Татарстан,Казань,Недвижимость,Комнаты,Продам,None,None,"Комната 8.2 м² в 5-к, 3/5 эт.",Продаю тёплую комнату с отличным новым ремонто...,900000.0,385,2017-03-21,Company,f1e11a40aaa06374bb8b1a551a85b6057754c06ecc0f0e...,2219.0,0.01635
8,34,2285e797fe2d,5977e301114d,Саратовская область,Саратов,Транспорт,Автомобили,С пробегом,Audi,80,"Audi 80, 1986","Отличный надёжный немец, на очень бодром ходу!...",65000.0,1,2017-03-25,Private,9dc0755c29addc765567c613087fbcabfd62f1555772f4...,1132.0,0.67548
9,38,f55eed4e831e,7e8eb4a5d1d5,Калининградская область,Гвардейск,Недвижимость,Коммерческая недвижимость,Продам,Складское помещение,None,Склад 70 м²,"Продается склад ГСМ, по адресу п. Истровка, Ка...",800000.0,6004,2017-03-28,Shop,None,NaN,0.06626


In [11]:
price['price']

0           40000.0
1           11000.0
2           18000.0
3          140000.0
4          100000.0
5         1748000.0
6          249000.0
7          900000.0
8           65000.0
9          800000.0
10        2300000.0
11        2350000.0
12          15000.0
13        2180000.0
14         150250.0
15          12000.0
16         740000.0
17          15000.0
18        1494000.0
19         250000.0
20          57000.0
21        1190000.0
22        1000000.0
23        1860000.0
24         107000.0
25         300000.0
26        1500000.0
27        7700000.0
28          85000.0
29         800000.0
            ...    
296150     410000.0
296151     620000.0
296152      80000.0
296153      18000.0
296154    5200000.0
296155    2370000.0
296156     800000.0
296157      10900.0
296158      50000.0
296159     320000.0
296160      30000.0
296161      19900.0
296162      13000.0
296163    1620000.0
296164      14500.0
296165      65000.0
296166    3800000.0
296167     420000.0
296168      70000.0
